In [ ]:
!pip3 install census
!pip3 install us

In [ ]:
conda list

In [1]:
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt


# Census API Key
c = Census(census_key, year=2013)

In [2]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
obesity_df = pd.read_csv(
    "./Resources/obesity.csv", dtype="object", encoding="utf-8")

# Visualize

obesity_df.head()

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Data_Value_Footnote,PopulationCount,GeoLocation,Lat,Long,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2017,HI,Hawaii,Honolulu,Census Tract,BRFSS,Unhealthy Behaviors,15003004600,Obesity among adults aged >=18 Years,%,...,NaN,"3,735","21.3510749539, -157.825340793",21.35107495,-157.8253408,UNHBEH,OBESITY,15003,15003004600,Obesity
1,2017,HI,Hawaii,Honolulu,Census Tract,BRFSS,Unhealthy Behaviors,15003001400,Obesity among adults aged >=18 Years,%,...,NaN,"2,550","21.2772209725, -157.806213514",21.27722097,-157.8062135,UNHBEH,OBESITY,15003,15003001400,Obesity
2,2017,GA,Georgia,Roswell,City,BRFSS,Unhealthy Behaviors,1367284,Obesity among adults aged >=18 Years,%,...,NaN,"88,346","34.0391384583, -84.3512225277",34.03913846,-84.35122253,UNHBEH,OBESITY,1367284,NaN,Obesity
3,2017,HI,Hawaii,Honolulu,Census Tract,BRFSS,Unhealthy Behaviors,15003010801,Obesity among adults aged >=18 Years,%,...,NaN,"2,969","21.4491051468, -157.761542724",21.44910515,-157.7615427,UNHBEH,OBESITY,15003,15003010801,Obesity
4,2017,GA,Georgia,Johns Creek,Census Tract,BRFSS,Unhealthy Behaviors,1342425-13121011425,Obesity among adults aged >=18 Years,%,...,NaN,"6,906","34.013011213, -84.2405514785",34.01301121,-84.24055148,UNHBEH,OBESITY,1342425,13121011425,Obesity


In [3]:
reduced_obesity_df = obesity_df[['StateDesc','CityName','Data_Value','Lat','Long']]
florida_obesity_df = reduced_obesity_df.loc[(reduced_obesity_df['StateDesc'] == "Florida" )]
florida_obesity_df

,StateDesc,CityName,Data_Value,Lat,Long
2374,Florida,Pembroke Pines,22.8,26.0169637,-80.35220159
7304,Florida,Deltona,33.4,28.90506005,-81.21342698
7305,Florida,Gainesville,28.5,29.68170274,-82.34527827
7890,Florida,Largo,28.7,27.90909077,-82.77142034
7960,Florida,Lauderhill,34.4,26.16055842,-80.22412783
...,...,...,...,...,...
23788,Florida,Tallahassee,36.4,30.4166907,-84.31106178
23789,Florida,Tallahassee,34.1,30.45839085,-84.30492785
23790,Florida,Tampa,37.9,28.04903942,-82.43451163
23793,Florida,Tallahassee,33.6,30.45489512,-84.25264599


In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations
locations = florida_obesity_df[["Lat", "Long"]].astype(float)
obesity_df = florida_obesity_df.dropna()
obesity_rate = florida_obesity_df["Data_Value"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=obesity_rate, 
                                 dissipating=False, max_intensity=20,
                                 point_radius = .0047)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
physical_df = pd.read_csv("./Resources/Physical_Activity_Years.csv", dtype="object", encoding="utf-8")

# Visualize

physical_df.head()

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,Data_Value_Footnote,PopulationCount,GeoLocation,Lat,Long,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
0,2017,IL,Illinois,Joliet,Census Tract,BRFSS,Unhealthy Behaviors,1738570-17197881200,No leisure-time physical activity among adults...,%,...,NaN,"4,124","(41.5450772077, -88.048940673)",41.54507721,-88.04894067,UNHBEH,LPA,1738570,17197881200,Physical Inactivity
1,2017,IL,Illinois,Peoria,Census Tract,BRFSS,Unhealthy Behaviors,1759000-17143003000,No leisure-time physical activity among adults...,%,...,NaN,"4,664","(40.7541602759, -89.6263008079)",40.75416028,-89.62630081,UNHBEH,LPA,1759000,17143003000,Physical Inactivity
2,2017,IL,Illinois,Chicago,Census Tract,BRFSS,Unhealthy Behaviors,1714000-17031640400,No leisure-time physical activity among adults...,%,...,NaN,"3,190","(41.7811783033, -87.7912473905)",41.7811783,-87.79124739,UNHBEH,LPA,1714000,17031640400,Physical Inactivity
3,2017,IL,Illinois,Naperville,Census Tract,BRFSS,Unhealthy Behaviors,1751622-17043846504,No leisure-time physical activity among adults...,%,...,NaN,"6,270","(41.7715785099, -88.1627076155)",41.77157851,-88.16270762,UNHBEH,LPA,1751622,17043846504,Physical Inactivity
4,2017,IL,Illinois,Chicago,Census Tract,BRFSS,Unhealthy Behaviors,1714000-17031170600,No leisure-time physical activity among adults...,%,...,NaN,"2,498","(41.9412942491, -87.8307017439)",41.94129425,-87.83070174,UNHBEH,LPA,1714000,17031170600,Physical Inactivity


In [6]:
reduced_nutrition_df = physical_df[['StateDesc','CityName','Unhealthy_Data_Value','Lat','Long']]
florida_nutrition_df = reduced_nutrition_df.loc[(reduced_nutrition_df['StateDesc'] == "Florida" )]
florida_nutrition_df

,StateDesc,CityName,Unhealthy_Data_Value,Lat,Long
23655,Florida,Hollywood,30.2,26.0177442,-80.23630177
23656,Florida,Cape Coral,26.7,26.69446722,-82.06089139
23657,Florida,Davie,26.7,26.05548252,-80.24197032
23658,Florida,Cape Coral,29.2,26.65964382,-82.01521992
23659,Florida,Hialeah,35.6,25.88056727,-80.33543093
...,...,...,...,...,...
25417,Florida,Tampa,27.8,27.98741705,-82.46487497
25419,Florida,West Palm Beach,35.4,26.76874095,-80.11958917
25420,Florida,Tallahassee,24,30.44278678,-84.30030985
25421,Florida,Tampa,34.9,28.01554165,-82.45788874


In [7]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations
locations = florida_nutrition_df[["Lat", "Long"]].astype(float)
nutrition_df = florida_nutrition_df.dropna()
nutrition_rate = florida_nutrition_df["Unhealthy_Data_Value"].astype(float)

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=nutrition_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius =0.0097)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

In [8]:
complete_data_df = pd.merge(florida_nutrition_df, florida_obesity_df, how="left", on=["Lat", "Long"])

# Remove rows missing data
complete_data_df = complete_data_df.dropna()

# Visualize
complete_data_df.head()

,StateDesc_x,CityName_x,Unhealthy_Data_Value,Lat,Long,StateDesc_y,CityName_y,Data_Value
0,Florida,Hollywood,30.2,26.0177442,-80.23630177,Florida,Hollywood,25.3
1,Florida,Cape Coral,26.7,26.69446722,-82.06089139,Florida,Cape Coral,28
2,Florida,Davie,26.7,26.05548252,-80.24197032,Florida,Davie,23.5
3,Florida,Cape Coral,29.2,26.65964382,-82.01521992,Florida,Cape Coral,29.5
4,Florida,Hialeah,35.6,25.88056727,-80.33543093,Florida,Hialeah,31.9


In [9]:
complete_data_df  = complete_data_df.rename(columns={"Data_Value": "Obesity_Data_Value"})
complete_data_df = complete_data_df[["StateDesc_x","CityName_x","Unhealthy_Data_Value","Obesity_Data_Value","Lat","Long",]]
complete_data_df.head()                                            

,StateDesc_x,CityName_x,Unhealthy_Data_Value,Obesity_Data_Value,Lat,Long
0,Florida,Hollywood,30.2,25.3,26.0177442,-80.23630177
1,Florida,Cape Coral,26.7,28,26.69446722,-82.06089139
2,Florida,Davie,26.7,23.5,26.05548252,-80.24197032
3,Florida,Cape Coral,29.2,29.5,26.65964382,-82.01521992
4,Florida,Hialeah,35.6,31.9,25.88056727,-80.33543093


In [11]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)
# Store 'Lat' and 'Lng' into  locations
locations = complete_data_df[["Lat", "Long"]].astype(float)
nutrition_df = complete_data_df.dropna()

nutrition_rate = complete_data_df["Unhealthy_Data_Value"].astype(float)

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=nutrition_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius =0.0087)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

In [12]:
params = {
    "location": 'location',             
    "radius": 2000,
    "type": 'gym',
    "key": gkey
}

# For Loop to go through each Zip code to find # of gyms
for index, row in complete_data_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Long"]
    
    params["location"] = f"{Lat},{Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    response = response.json()
    complete_data_df.loc[index, "Gym"] = len(response["results"])
    
    
complete_data_df

,StateDesc_x,CityName_x,Unhealthy_Data_Value,Obesity_Data_Value,Lat,Long,Gym
0,Florida,Hollywood,30.2,25.3,26.0177442,-80.23630177,14.0
1,Florida,Cape Coral,26.7,28,26.69446722,-82.06089139,0.0
2,Florida,Davie,26.7,23.5,26.05548252,-80.24197032,18.0
3,Florida,Cape Coral,29.2,29.5,26.65964382,-82.01521992,0.0
4,Florida,Hialeah,35.6,31.9,25.88056727,-80.33543093,20.0
...,...,...,...,...,...,...,...
1458,Florida,Tampa,27.8,29.9,27.98741705,-82.46487497,10.0
1459,Florida,West Palm Beach,35.4,32.5,26.76874095,-80.11958917,7.0
1460,Florida,Tallahassee,24,24.5,30.44278678,-84.30030985,16.0
1461,Florida,Tampa,34.9,35.2,28.01554165,-82.45788874,8.0


In [13]:
#Setting paramaeters to find # of liquor stores per Lat & Long
params = {
    "location": 'location',             
    "radius": 2000,
    "type": 'liquor_store',
    "key": gkey
}

# For Loop to go through each Zip code to find # of liquor stores
for index, row in ccomplete_data_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Long"]
    
    params["location"] = f"{Lat},{Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    response = response.json()
    complete_data_df.loc[index, "Liquor Store"] = len(response["results"])

complete_data_df  

NameError: name 'ccomplete_data_df' is not defined

In [ ]:
params = {
    "location": 'location',             
    "radius": 2000,
    "type": 'supermarket',
    "key": gkey
}
for index, row in test_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Long"]
    
    params["location"] = f"{Lat},{Lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params)
    response = response.json()
    complete_data_df.loc[index, "Super Markets"] = len(response["results"])

complete_data_df   
#print(json.dumps(response, indent=4, sort_keys=True))